# 欠損値の補完
欠損値の補完には基本的に連続値だと「平均値」「中央値」でカテゴリだと「最頻値」などの代表値を使って埋めたり時系列データだと「移動平均」を用いて埋めることがある。しかしそれらが必ず正確とも言えず、ある機械学習の資格試験でも機械学習による予測で穴埋めすることの正当性に触れている。

# ライブラリのインポート

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import GradientBoostingRegressor as GBR

# データの読み込み

In [ ]:
df = pd.read_csv("irisNaN.csv")

# カラムに欠損があるか確認

In [ ]:
df.isnull().any()

category             True
sepal length (cm)    True
sepal width (cm)     True
petal length (cm)    True
petal width (cm)     True
dtype: bool

# 各カラムの欠損の数を確認

In [ ]:
df.isnull().sum()

category             3
sepal length (cm)    3
sepal width (cm)     3
petal length (cm)    3
petal width (cm)     3
dtype: int64

最初は欠損の少ないカラムから埋めていく(今回数は同じにしたため順を追うが、重要度を計測して確実に埋めるのも良い)

また、名義尺度のカラムにあるカテゴリの数が3種類以上ある場合はワンホットエンコーディングを使う

In [ ]:
columns = list(df.columns)
columns

['category',
 'sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

欠損の埋め込みを開始する　今回はcategory

In [ ]:
tmp_col = columns
y_name = "category"
tmp_col.remove(y_name)
tmp_col

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

目的変数以外の欠損のある場所を削除する

In [ ]:
df_tmp = df.dropna(subset=tmp_col)
df_tmp.isnull().sum()

category             3
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
dtype: int64

In [ ]:
df_train = df_tmp.query("category==category")
df_test = df_tmp.query("category!=category")

In [ ]:
df_train.head()

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.0,5.1,3.5,1.4,0.2
1,0.0,4.9,3.0,1.4,0.2
2,0.0,4.7,3.2,1.3,0.2
3,0.0,4.6,3.1,1.5,0.2
4,0.0,5.0,3.6,1.4,0.2


In [ ]:
df_test.head()

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
49,NaN,5.0,3.3,1.4,0.2
99,NaN,5.7,2.8,4.1,1.3
149,NaN,5.9,3.0,5.1,1.8


categoryは名義尺度(カテゴリ)であるため分類で欠損値を埋める

In [ ]:
y_train = df_train[y_name].values
x_train = df_train.drop(y_name, axis=1).values
x_test = df_test.drop(y_name, axis=1).values
model = GBC()#ハイパーパラメータチューニングは省略
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
y_pred

array([0., 1., 2.])

numpyの横結合で予測値と説明変数を結合する

In [ ]:
import numpy as np
y_pred = y_pred.reshape(-1, 1)
pred = np.hstack((y_pred, x_test))
df_pred = pd.DataFrame(pred)
df_pred.columns = df.columns
df_pred

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.0,5.0,3.3,1.4,0.2
1,1.0,5.7,2.8,4.1,1.3
2,2.0,5.9,3.0,5.1,1.8


データフレームの結合

In [ ]:
df_before = df.dropna(subset=[y_name])
df_new = pd.concat([df_before, df_pred])

欠損値の個数を確認

In [ ]:
df_new.isnull().sum()

category             0
sepal length (cm)    3
sepal width (cm)     3
petal length (cm)    3
petal width (cm)     3
dtype: int64

categoryから欠損が消えたことを確認できた。

連続値の場合は回帰系の機械学習を用いて穴埋めを行う

In [ ]:
y_name = "sepal length (cm)"
tmp_col = list(df.columns)
tmp_col.remove(y_name)
tmp_col

['category', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

In [ ]:
df_tmp = df_new.dropna(subset=tmp_col)
df_tmp.isnull().sum()

category             0
sepal length (cm)    3
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
dtype: int64

categoryはカテゴリ変数であるためワンホットエンコーディングを行います

今回はカラム名にスペースが入っているためqueryメソッドは使いません。

In [ ]:
df_d = pd.get_dummies(df_tmp, columns=["category"])
df_train = df_d.dropna()
df_test = df_d[df_d[y_name].isnull()]
df_test2 = df_tmp[df_tmp[y_name].isnull()]
df_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),category_0.0,category_1.0,category_2.0
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [ ]:
df_test.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),category_0.0,category_1.0,category_2.0
48,NaN,3.7,1.5,0.2,1,0,0
98,NaN,2.5,3.0,1.1,0,1,0
148,NaN,3.4,5.4,2.3,0,0,1


In [ ]:
y_train = df_train[y_name].values
y_test = df_test[y_name].values
x_train = df_train.drop([y_name], axis=1).values
x_test = df_test.drop([y_name], axis=1).values
x_test2 = df_test2.drop([y_name], axis=1).values

In [ ]:
model = GBR()#ハイパーパラメータチューニングは省略
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred

array([5.27932953, 5.02225485, 6.60259939])

In [ ]:
tmp = np.hstack((x_test2[:, 0].reshape(-1, 1), y_pred.reshape(-1, 1)))
pred = np.hstack((tmp, x_test2[:, 1:]))
df_pred = pd.DataFrame(pred)
df_pred.columns = df.columns
df_pred

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.0,5.279330,3.7,1.5,0.2
1,1.0,5.022255,2.5,3.0,1.1
2,2.0,6.602599,3.4,5.4,2.3


欠損を修正したdf_newを用いることに注意

In [ ]:
df_before = df_new.dropna(subset=[y_name])
df_new = pd.concat([df_before, df_pred])
df_new.head()

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.0,5.1,3.5,1.4,0.2
1,0.0,4.9,3.0,1.4,0.2
2,0.0,4.7,3.2,1.3,0.2
3,0.0,4.6,3.1,1.5,0.2
4,0.0,5.0,3.6,1.4,0.2


In [ ]:
df_new.isnull().sum()

category             0
sepal length (cm)    0
sepal width (cm)     3
petal length (cm)    3
petal width (cm)     3
dtype: int64

categoryとsepal length (cm)で欠損値が無くなったことが確認できた

残りの部分はsepal length (cm)と同じやり方で消していく。